In [10]:
import torchvision
import torch.nn as nn
import torch.nn.functional as nnf
import torch
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms.functional import to_tensor, normalize
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from torchvision import transforms, models
from torchvision.datasets import Food101
from torchvision.models import ResNet50_Weights
import os
import shutil
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.get_device_name(0))

def test_acc(net: nn.Module, test_loader: DataLoader):

  net.to(device)
  net.eval()
  
  total = 0
  correct = 0
  top2_correct = 0

  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    total += labels.size(0)

    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    correct += (predicted == labels).sum().item()

    _, predicted2 = torch.topk(outputs, 2, dim=1)  # Get top 2 predictions
    top2_correct += ((predicted2 == labels.view(-1, 1)) | (predicted2[:, 1].view(-1, 1) == labels.view(-1, 1))).sum().item()

  return (correct / total * 100, top2_correct / total * 100)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        resnet = nn.Sequential(*list(resnet.children())[:-1])
        for param in resnet.parameters():
            param.requires_grad = False


        self.model = nn.Sequential(
            resnet,
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.25),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.1),
            nn.Linear(128, 103)
        )

    def forward(self, x):
        return self.model(x)

cuda
NVIDIA GeForce RTX 4060 Laptop GPU


In [11]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dir = "C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/Facultate/IAVA/Proiect/Food Classifier Final/cross_validation/google_new"

test_dataset = datasets.ImageFolder(test_dir, transform=transform)

batch_size = 32
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [12]:
model = Model()

start = time.time()
for i in range(1, 16):

    state_dict = torch.load(f'model103_epoch{i}.pkl')
    model.load_state_dict(state_dict)

    accuracy, top2accuracy = test_acc(model, test_loader)

    print(f'Epoch {i}: accuracy - {accuracy:.3f}%, top 2 accuracy - {top2accuracy:.3f}%')

print(f'Testing duration: {time.time() - start}')

Epoch 1: accuracy - 33.551%, top 2 accuracy - 57.235%
Epoch 2: accuracy - 36.751%, top 2 accuracy - 61.679%
Epoch 3: accuracy - 34.598%, top 2 accuracy - 58.202%
Epoch 4: accuracy - 40.049%, top 2 accuracy - 66.835%
Epoch 5: accuracy - 37.916%, top 2 accuracy - 63.714%
Epoch 6: accuracy - 39.684%, top 2 accuracy - 66.607%
Epoch 7: accuracy - 41.067%, top 2 accuracy - 68.267%
Epoch 8: accuracy - 41.817%, top 2 accuracy - 66.884%
Epoch 9: accuracy - 40.741%, top 2 accuracy - 65.373%
Epoch 10: accuracy - 36.030%, top 2 accuracy - 62.242%
Epoch 11: accuracy - 42.005%, top 2 accuracy - 67.625%
Epoch 12: accuracy - 38.805%, top 2 accuracy - 64.919%
Epoch 13: accuracy - 39.052%, top 2 accuracy - 64.454%
Epoch 14: accuracy - 36.681%, top 2 accuracy - 62.519%
Epoch 15: accuracy - 41.047%, top 2 accuracy - 67.062%
Testing duration: 4281.951848745346
